In [14]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math, Latex

In [2]:
np.random.seed(0)

In [3]:
# the code is [[4, 2, 2]]
# we encode 2 logical qubits into 4 physical qubits

Hx = np.array([
    [1, 1, 1, 1]
], dtype=int)

Hz = np.array([
    [1, 1, 1, 1]
], dtype=int)

Lx = np.array([[1, 1, 0, 0], [1, 0, 1, 0]])
Lz = np.array([[1, 0, 0, 1], [0, 1, 1, 0]])

print(f"We have {Hx.shape[1]} data qubits")
print(f"We have {Hx.shape[0]} X-Stabilizers")
print(f"We have {Hz.shape[0]} Z-Stabilizers")
print(f"We have {Hx.shape[1] + Hx.shape[0] + Hz.shape[0]} physical qubits")
print(f"We have {Lx.shape[0]} logical qubits")


We have 4 data qubits
We have 1 X-Stabilizers
We have 1 Z-Stabilizers
We have 6 physical qubits
We have 2 logical qubits


In [ ]:
orthogonality = (Hx @ Hz.T) % 2
display(Math(r'\text{Orthogonality test: } H_X \cdot H_Z^T \text{ mod 2 gave}'))

Orthogonality test: $H_X \cdot H_Z^T$ mod 2 gave: 0


<IPython.core.display.Math object>